In [ ]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database

importlib.reload(import_default)
importlib.reload(import_database)

from import_default import *
from import_database import *

In [5]:
date_series = get_mongo_collection(ref_db, 'date_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')

In [ ]:
projection = {"_id": 0,"date": 1,"ticker": 1,"open": 1,"high": 1,"low": 1,"close": 1, "volume": 1, "cap": 1}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

today_stock_df = get_mongo_collection(stock_db, "today_stock", projection=projection)
history_stock_df = get_mongo_collection(stock_db, "history_stock", projection=projection)
full_stock_df = pd.concat([today_stock_df, history_stock_df], axis=0, ignore_index=True)

full_ticker_df = pd.concat([full_index_df, full_stock_df], axis=0, ignore_index=True)

In [50]:
full_stock_df['industry'] = full_stock_df['ticker'].map(full_stock_classification_df.set_index('ticker')['industry_name'])
full_stock_df['fullname_industry'] = full_stock_df['ticker'].map(full_stock_classification_df.set_index('ticker')['fullname_industry'])
bds_stock_df = full_stock_df[full_stock_df['industry'] == 'bds']

bds_stock_list = []
for stock in bds_stock_df['ticker'].unique():
    temp_df = bds_stock_df[bds_stock_df['ticker'] == stock]
    temp_df['cap'] = temp_df['close'] * (temp_df['cap'].iloc[0] / temp_df['close'].iloc[0])
    max_cap = temp_df['cap'].max()
    current_cap = temp_df[temp_df['date'] == date_series['date'].max()]['cap'].values[0]
    bds_stock_list.append([stock, max_cap, current_cap])

bds_cap_df = pd.DataFrame(bds_stock_list, columns=['ticker', 'max_cap', 'current_cap'])
bds_cap_df.to_excel('../output/bds_cap_v3.xlsx', index=False)